# OpenMASTER - correlations

-------------------------

In [1]:
import pyomo.environ as pyo
import openMASTER
import numpy as np
import itertools
import pandas as pd

### Define the abstract model

In [ ]:
m = openMASTER.make_model()
m

### Model data upload

* If you haven't created the .csv files, please:
    * Be aware the openMASTER_Data.xlsx file has to be downloaded using git-lfs or the following link:
        https://github.com/IIT-EnergySystemModels/openMASTER/raw/main/data/input/openMASTER_Data.xlsx?download=
    * Run the first line of code in this cell, which will both create the .csv files and load them into the DataPortal (this whole function takes several minutes)

* On the contrary, if you have already created the .csv files from the Excel file and haven't changed them, you can directly go on to the second line of code. This will save some minutes.

In any case, add "#" in front of the line you are not running.

In [ ]:
data = openMASTER.load_dataportal_from_excel('../data/input/openMASTER_Data.xlsx')
#data = openMASTER.load_dataportal_from_csv()

### Principal components analysis (PCA)

In [ ]:
csv_mean   = '..\data\input\mean.csv'
df_mean    = pd.read_csv(csv_mean, delimiter=';', header=[0])
X_mean     = df_mean.values


#csv_data   = '..\data\input\costs_correl.csv'
#df_data    = pd.read_csv(csv_data, delimiter=';', header=[0])
#X_data     = df_data.values
#X_o        = X_data - np.mean(X_data, axis=0)

#X_z        = X_o / np.std (X_o, axis=0)
#X_z        = np.nan_to_num(X_z, nan=0.0)

# Encontrar las columnas con valores NaN en X_z
#columns_with_nan = np.isnan(X_z).any(axis=0)

# Eliminar las columnas con valores NaN en X_z
#X_z_clean = X_z[:, ~columns_with_nan]

# Eliminar las mismas columnas en X_o
#X_o_clean = X_o[:, ~columns_with_nan]

#print(X_o)

csv_covar  = '..\data\input\covar.xlsx'
sheet_name = 'covar'
df_covar   = pd.read_excel(csv_covar, sheet_name=sheet_name, index_col=0)
X_covar    = df_covar.values


csv_data  = '..\data\input\covar.xlsx'
sheet_name = 'data'
df_data   = pd.read_excel(csv_data, sheet_name=sheet_name, index_col=0)
X_hist    = df_data.values


f_set   = data['f']
unc_set = data['sUnc']


# Compute the mean for each column

#s_bar_dict = {unc: s_bar[i-1] for i, unc in enumerate(unc_set)}

# Set a seed for reproducibility
np.random.seed(93)  # You can use any integer value as the seed

# Calcular las varianzas de la serie histórica
variances_hist = np.var(X_hist, axis=0)

# Escalar la matriz de covarianza utilizando las varianzas de la serie histórica
scaled_covar = X_covar * variances_hist

# Generar los datos sintéticos utilizando la media y la matriz de covarianza escalada
X = np.random.multivariate_normal(np.ravel(X_mean), scaled_covar, size=10000)


# Ensure all values are non-negative
#X = np.maximum(X, 0)

X_o = X - np.mean(X, axis=0)


s_bar_dict = {unc: np.ravel(X_mean)[i] for i, unc in enumerate(unc_set)}


from sklearn.decomposition import PCA
#we compute the singular value decomposition

#we use eigvals, eigvecs so they are not ordered in decreasing order
#eigvals, eigvecs = np.linalg.eig(np.cov(X_o.T))

#eigvecs_real = np.real(eigvecs)
#eigvecs_imag = np.imag(eigvecs)

#eigvals_real = np.real(eigvals)
#eigvals_imag = np.imag(eigvals)

cov_matrix = np.cov(X_o.T)


u, s, vh = np.linalg.svd(cov_matrix, full_matrices=True)
#u, s, vh = np.linalg.svd(X_covar, full_matrices=True)

#pca1 = PCA()
#pca1.fit(X_o)

#u2, s2, vh2 = np.linalg.svd(np.cov(X_z_clean.T), full_matrices=True)
#pca2 = PCA()
#pca2.fit(X_z_clean)

# Comparar los componentes principales obtenidos de ambos métodos
#print("Componentes principales obtenidos por el método 1:")
#print(pca1.components_)
#print("\nComponentes principales obtenidos por el método 2:")
#print(pca2.components_)

# Comprobación de similitud
#similarity = np.allclose(pca1.components_, pca2.components_)
#print("\n¿Los componentes principales son similares?", similarity)


##np.linalg.svd --> Singular Value Decomposition:
##np.cov --> estimate the covariance matrix
##u is eigenvector matrix and s are the eigenvalues

#prueba = np.dot(X_o, u)

#now we compute the vectors needed to build the uncertainty region
w_max = np.max(np.dot(X_o, u) / np.linalg.norm(u, axis=0), 0)
#w_max = np.where((w_max > -1e-03) & (w_max < 1e-03), 0, w_max)
w_max_dict = {f_i: w_max[i] for i, f_i in enumerate(f_set)}
#w_max_dict = {f_i: w_max[i-1] for i, f_i in enumerate(f_set)}

w_min = np.min(np.dot(X_o, u) / np.linalg.norm(u, axis=0), 0)
#w_min = np.where((w_min > -1e-03) & (w_min < 1e-03), 0, w_min)
w_min_dict = {f_i: w_min[i] for i, f_i in enumerate(f_set)}
#w_min_dict = {f_i: w_min[i-1] for i, f_i in enumerate(f_set)}

alpha_up = (w_max / np.linalg.norm(u, axis=0)) * u
alpha_up_dict = {(unc, f_i): alpha_up[i, j] for i, unc in enumerate(unc_set) for j, f_i in enumerate(f_set)}

alpha_do = (w_min / np.linalg.norm(u, axis=0)) * u
alpha_do_dict = {(unc, f_i): alpha_do[i, j] for i, unc in enumerate(unc_set) for j, f_i in enumerate(f_set)}

rest = ((w_max + w_min) / (2 * np.linalg.norm(u, axis=0))) * u
rest_dict = {(unc, f_i): rest[i, j] for i, unc in enumerate(unc_set) for j, f_i in enumerate(f_set)}


data._data[None][m.pS.name]        = s_bar_dict
data._data[None][m.pW_max.name]    = w_max_dict
data._data[None][m.pW_min.name]    = w_min_dict
data._data[None][m.pAlpha_up.name] = alpha_up_dict
data._data[None][m.pAlpha_do.name] = alpha_do_dict
data._data[None][m.pRest.name]     = rest_dict

print(alpha_up_dict)

### Create the instance of the abstract model 

In [ ]:
instance = m.create_instance(data)

In [ ]:
#instance.write('instance.lp', io_options={'symbolic_solver_labels': True})

In [ ]:
#data._data[None][m.pCECapex.name]
#instance.pS.pprint()

### Solve the model instance

To solve the model instance, please select a solver within the Pyomo SolverFactory. Please note that any solver has to be previously installed.

In [ ]:
solver = pyo.SolverFactory('gurobi')

solver_results = solver.solve(instance, keepfiles=False, tee=True)

### **Results**

Extract information on variables through the model output to .csv files

In [ ]:
path        = "../data/input/openMASTER_Data.xlsx"
output_path = "../data/tmp/output"
sheetname   = "Output"

d_vars_from_instance = openMASTER.export_model_to_csv(path, output_path, sheetname, instance)

Loading variable information from .csv to a dictionary containing all outputs

In [ ]:
d_vars = openMASTER.import_results_from_csv(output_path)

In [ ]:
import os

vQSTInTE_results = d_vars['vQSTInTE']
vQSTInTE_sum = vQSTInTE_results.groupby(['sYear', 'sTE', 'sST']).sum()

# Define the output file path
output_file = os.path.join(output_path, "vQSTInTE_sum.csv")

# Save the DataFrame as a CSV file
vQSTInTE_sum.to_csv(output_file)


Checks

In [ ]:
vQESNS_results = d_vars['vQESNS']
vQESNS_sum = vQESNS_results['vQESNS'].sum()
print("Sum of vQESNS:    ", vQESNS_sum)

In [ ]:
vEmiCO2CapTraExc_sum    = d_vars['vEmiCO2CapTraExc'].sum()
vEmiCO2CapEleExc_sum    = d_vars['vEmiCO2CapEleExc'].sum()
vEmiCO2CapIndTEExc_sum  = d_vars['vEmiCO2CapIndTEExc'].sum()
vEmiCO2CapIndProExc_sum = d_vars['vEmiCO2CapIndProExc'].sum()
vEmiCO2CapOthExc_sum    = d_vars['vEmiCO2CapOthExc'].sum()
vEmiCO2CapRefExc_sum    = d_vars['vEmiCO2CapRefExc'].sum()

print("Sum of vEmiCO2CapTraExc:    ", vEmiCO2CapTraExc_sum)
print("Sum of vEmiCO2CapEleExc:    ", vEmiCO2CapEleExc_sum)
print("Sum of vEmiCO2CapIndTEExc:  ", vEmiCO2CapIndTEExc_sum)
print("Sum of vEmiCO2CapIndProExc: ", vEmiCO2CapIndProExc_sum)
print("Sum of vEmiCO2CapOthExc:    ", vEmiCO2CapOthExc_sum)
print("Sum of vEmiCO2CapRefExc:    ", vEmiCO2CapRefExc_sum)